**Full name : El ghazi Loubna** <br/>


Part 1 : Classification/ Regression

In [80]:
import torch
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    
import scrapy
from scrapy.crawler import CrawlerProcess
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.translate.bleu_score import sentence_bleu
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, GRU, Dense, Embedding
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords

In [82]:
def generate_scores(texts):
    scores = []
    for text in texts:
        score = 5.0  # Score initial
        
        keywords = ['آستشهاد','إسرائيل','غزو','حرب','قتال','حماس', 'فلسطين', 'غزة', 'صراع', 'مقاومة']
        for keyword in keywords:
            if keyword in text:
                score += 1.0  
        negative_words = ['زهرة', 'قوس قزح', 'سلام']
        for word in negative_words:
            if word in text:
                score -= 1.0  
        scores.append(score)
    return scores



In [81]:
class AlJazeeraSpider(scrapy.Spider):
    name = "aljazeera"
    
    start_urls = ['https://www.aljazeera.net/news/']
    def __init__(self, *args, **kwargs):
        super(AlJazeeraSpider, self).__init__(*args, **kwargs)
        self.texts = []

    
    def parse(self, response):
        for article_link in response.css('a.link'):
            yield response.follow(article_link, callback=self.parse_article)
    
    def parse_article(self, response):
        article_title = response.css('h1.article-title::text').get()
        if article_title:
            self.texts.append(article_title.strip())
   
    def run_spider():
        process = CrawlerProcess()
        process.crawl(AlJazeeraSpider)
        process.start()
        return process.spiders[AlJazeeraSpider.name].texts

        # Run the spider to get the texts
        texts = run_spider()

In [83]:
scores = generate_scores(texts)

In [84]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import ISRIStemmer

def preprocess(text):
    # Remove non-Arabic characters and normalize the text
    cleaned_text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    cleaned_text = arabic_reshaper.reshape(cleaned_text)
    cleaned_text = get_display(cleaned_text)
    
    # Tokenization
    tokens = word_tokenize(cleaned_text)
    
    # Remove stop words
    stop_words = set(stopwords.words('arabic'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Stemming using ISRIStemmer
    stemmer = ISRIStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    
    # Join tokens into a single string
    preprocessed_text = ' '.join(stemmed_tokens)
    
    return preprocessed_text


In [85]:
def prepare_data(texts, scores):
    preprocessed_texts = [preprocess(text) for text in texts]
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(preprocessed_texts)
    sequences = tokenizer.texts_to_sequences(preprocessed_texts)
    X = pad_sequences(sequences)
    y = np.array(scores)
    return X, y

In [86]:
def build_model(word_index):
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, 128))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    return model

In [87]:
def train_model(X_train, y_train, X_val, y_val):
    model = build_model(np.max(X_train))
    early_stopping = EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])
    return model, history

In [88]:

# Preprocess and prepare data
X, y = prepare_data(texts, scores)


In [92]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


ValueError: With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [91]:
model, history = train_model(X_train, y_train, X_temp, y_temp)

TypeError: object of type 'numpy.int32' has no len()

In [73]:
def evaluate_model(model, X_test, y_test):
    y_pred = np.argmax(model.predict(X_temp), axis=-1)
    accuracy = accuracy_score(y_temp, y_pred)
    precision = precision_score(y_temp, y_pred)
    recall = recall_score(y_temp, y_pred)
    f1 = f1_score(y_temp, y_pred)
    return accuracy, precision, recall, f1

In [74]:
accuracy, precision, recall, f1 = evaluate_model(model, X_test, y_test)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

NameError: name 'X_test' is not defined

In [ ]:
# Calculate BLEU score (Example)
reference = ["reference sentence"]
candidate = "generated sentence"
bleu_score = sentence_bleu(reference, candidate)
print("BLEU Score:", bleu_score)